# "数据集：银行产品推荐
组队完成比赛
数据处理，模型选择，训练并得出预测结果
分享交流你们的经验

## 1. Input data
这里加了一列序号列

In [1]:
import pandas as pd
import numpy as np
import os

data_train = pd.read_csv('/Users/huangzhan/Documents/开课吧/RS/Chap4.1-预测全家桶与机器学习四大神器/L12/santander-product-recommendation/train.csv',index_col=None)
data_train.insert(0, 'ID', range(len(data_train)))

data_test = pd.read_csv('/Users/huangzhan/Documents/开课吧/RS/Chap4.1-预测全家桶与机器学习四大神器/L12/santander-product-recommendation/test.csv',index_col=None)
data_test.insert(0, 'ID', range(len(data_test)))

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Check data 

for NAN(.isnull().any()/print(train.isna().sum())), repeated,type(.info)

In [2]:
data_train.isna().sum()
#data_train.isnull().any()

ID                            0
fecha_dato                    0
ncodpers                      0
ind_empleado               1146
pais_residencia            1146
sexo                       1146
age                           0
fecha_alta                 1146
ind_nuevo                  1146
antiguedad                    0
indrel                     1146
ult_fec_cli_1t           532841
indrel_1mes                6150
tiprel_1mes                6150
indresi                    1146
indext                     1146
conyuemp                 533767
canal_entrada              7661
indfall                    1146
tipodom                    1146
cod_prov                   3769
nomprov                    3769
ind_actividad_cliente      1146
renta                    106821
segmento                   7782
ind_ahor_fin_ult1             0
ind_aval_fin_ult1             0
ind_cco_fin_ult1              0
ind_cder_fin_ult1             0
ind_cno_fin_ult1              0
ind_ctju_fin_ult1             0
ind_ctma

In [3]:
data_test.isna().sum()
#data_test.isnull().any()

ID                           0
fecha_dato                   0
ncodpers                     0
ind_empleado                 0
pais_residencia              0
sexo                         0
age                          0
fecha_alta                   0
ind_nuevo                    0
antiguedad                   0
indrel                       0
ult_fec_cli_1t           29159
indrel_1mes                  0
tiprel_1mes                  0
indresi                      0
indext                       0
conyuemp                 29162
canal_entrada                2
indfall                      0
tipodom                      0
cod_prov                   139
nomprov                    139
ind_actividad_cliente        0
renta                     6757
segmento                     2
dtype: int64

In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533770 entries, 0 to 533769
Data columns (total 49 columns):
ID                       533770 non-null int64
fecha_dato               533770 non-null object
ncodpers                 533770 non-null int64
ind_empleado             532624 non-null object
pais_residencia          532624 non-null object
sexo                     532624 non-null object
age                      533770 non-null object
fecha_alta               532624 non-null object
ind_nuevo                532624 non-null float64
antiguedad               533770 non-null object
indrel                   532624 non-null float64
ult_fec_cli_1t           929 non-null object
indrel_1mes              527620 non-null object
tiprel_1mes              527620 non-null object
indresi                  532624 non-null object
indext                   532624 non-null object
conyuemp                 3 non-null object
canal_entrada            526109 non-null object
indfall                  532624 n

In [5]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29162 entries, 0 to 29161
Data columns (total 25 columns):
ID                       29162 non-null int64
fecha_dato               29162 non-null object
ncodpers                 29162 non-null int64
ind_empleado             29162 non-null object
pais_residencia          29162 non-null object
sexo                     29162 non-null object
age                      29162 non-null int64
fecha_alta               29162 non-null object
ind_nuevo                29162 non-null float64
antiguedad               29162 non-null int64
indrel                   29162 non-null float64
ult_fec_cli_1t           3 non-null object
indrel_1mes              29162 non-null float64
tiprel_1mes              29162 non-null object
indresi                  29162 non-null object
indext                   29162 non-null object
conyuemp                 0 non-null float64
canal_entrada            29160 non-null object
indfall                  29162 non-null object
tipodo

两个数据集都有共同的前23列

train中以下列都有NAN且ult_fec_cli_1t、conyuemp、ind_actividad_cliente和renta 列NAN的数量非常多：
ind_empleado               1146 
pais_residencia            1146
sexo                       1146
fecha_alta                 1146
ind_nuevo                  1146
indrel                     1146
ult_fec_cli_1t           532841
indrel_1mes                6150
tiprel_1mes                6150
indresi                    1146
indext                     1146
conyuemp                 533767
canal_entrada              7661
indfall                    1146
tipodom                    1146
cod_prov                   3769
nomprov                    3769
ind_actividad_cliente      1146
renta                    106821
segmento                   7782

ind_nomina_ult1             717
ind_nom_pens_ult1           717

---

test中以下列都有NAN且ult_fec_cli_1t、conyuemp和renta列NAN的数量非常多：
ult_fec_cli_1t           29159
conyuemp                 29162
cod_prov                   139
nomprov                    139
renta                     6757

.info()后会发现其中许多列是object和float64类型

所以，我们首先删去ult_fec_cli_1t、conyuemp两列

In [6]:
#删去ult_fec_cli_1t、conyuemp两列
del data_train['ult_fec_cli_1t']
del data_test['ult_fec_cli_1t']
del data_train['conyuemp']
del data_test['conyuemp']

## 2. data pre-process
losing(.dropna(axis = 1))/fill-in(.fillna(method = 'ffill', axis = 0))/average(.interpolate(method = 'linear', axis = 0)) the nulls

label-encoder/one-hot

normalized

In [7]:
#here dropna 
dtrain_d = data_train.dropna(axis = 0)
dtrain_d.info()
dtest_d = data_test.dropna(axis = 0)
dtest_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424488 entries, 0 to 533769
Data columns (total 47 columns):
ID                       424488 non-null int64
fecha_dato               424488 non-null object
ncodpers                 424488 non-null int64
ind_empleado             424488 non-null object
pais_residencia          424488 non-null object
sexo                     424488 non-null object
age                      424488 non-null object
fecha_alta               424488 non-null object
ind_nuevo                424488 non-null float64
antiguedad               424488 non-null object
indrel                   424488 non-null float64
indrel_1mes              424488 non-null object
tiprel_1mes              424488 non-null object
indresi                  424488 non-null object
indext                   424488 non-null object
canal_entrada            424488 non-null object
indfall                  424488 non-null object
tipodom                  424488 non-null float64
cod_prov                

In [8]:
#here fillna will fill NAN with the last non-null number(method = 'ffill', axis = 0) or a certain number
#using the last non-null number will cause bais
#dtrain_f = data_train.fillna(method = 'ffill', axis = 0)
#print(dtrain_f.info())
#dtest_f = data_test.fillna(method = 'ffill', axis = 0)
#print(dtest_f.info())

In [9]:
#here we choose interpolate
#dtrain = data_train.interpolate(method = 'linear', axis = 0)
#dtrain.info()
#dtrain.isna().sum()

**在处理NAN值时，我们采取了三种办法

dropna:
Index: 424488 entries, 2015-01-28 to 2016-04-28
Data columns (total 45 columns)

优点：剩下的数据都是有效的
缺点：数据损失

fillna:
Index: 533770 entries, 2015-01-28 to 2016-04-28
Data columns (total 45 columns)

优点：数据保留较全
缺点：NAN被前一位非NAN替代，有偏差

interpolate
Index: 533770 entries, 2015-01-28 to 2016-04-28
Data columns (total 45 columns)

优点：数据保留较全
缺点：type为object的其中一部分列依然存在缺失值

所以这里我们优先考虑第一种办法

In [10]:
print(dtrain_d.head())
print(dtest_d.head())

   ID  fecha_dato  ncodpers ind_empleado pais_residencia sexo  age  \
0   0  2015-01-28   1050676            N              ES    V   31   
1   1  2015-01-28   1050662            N              ES    V   37   
2   2  2015-01-28   1050652            N              ES    H   31   
3   3  2015-01-28   1050489            N              ES    H   23   
5   5  2015-01-28   1050497            N              ES    H   23   

   fecha_alta  ind_nuevo antiguedad  ...  ind_hip_fin_ult1 ind_plan_fin_ult1  \
0  2012-08-10        0.0         35  ...                 0                 0   
1  2012-08-10        0.0         35  ...                 0                 0   
2  2012-08-10        0.0         35  ...                 0                 0   
3  2012-08-10        0.0         35  ...                 0                 0   
5  2012-08-10        0.0         35  ...                 0                 0   

  ind_pres_fin_ult1 ind_reca_fin_ult1 ind_tjcr_fin_ult1 ind_valo_fin_ult1  \
0                 0  

现在来处理type为object的部分：
在train中：
ind_empleado             532624 non-null object
pais_residencia          532624 non-null object
sexo                     532624 non-null object
age                      533770 non-null object
fecha_alta               532624 non-null object
antiguedad               533770 non-null object
indrel_1mes              527620 non-null object
tiprel_1mes              527620 non-null object
indresi                  532624 non-null object
indext                   532624 non-null object
canal_entrada            526109 non-null object
indfall                  532624 non-null object
nomprov                  530001 non-null object
segmento                 525988 non-null object

在test中：
ind_empleado             29162 non-null object
pais_residencia          29162 non-null object
sexo                     29162 non-null object
fecha_alta               29162 non-null object
tiprel_1mes              29162 non-null object
indresi                  29162 non-null object
indext                   29162 non-null object
canal_entrada            29162 non-null object
indfall                  29162 non-null object
nomprov                  29162 non-null object
segmento                 29162 non-null object

对比发现，其中age, antiguedad, indrel_1mes在train中为object，但在test中不是

## 2.1 age, antiguedad 和 indrel_1mes

In [11]:
#查看age, antiguedad, indrel_1mes和indrel_1mes
print(dtrain_d['age'])
print(dtest_d['age'])
print(dtrain_d['antiguedad'])
print(dtest_d['antiguedad'])
print(dtrain_d['indrel_1mes'])
print(dtest_d['indrel_1mes'])
#print(dtrain_d['age'].unique())
#print(dtrain_d['antiguedad'].unique())
#print(dtrain_d['indrel_1mes'].unique())

0          31
1          37
2          31
3          23
5          23
         ... 
533765     41
533766     40
533767     65
533768     65
533769     57
Name: age, Length: 424488, dtype: object
0        42
1        49
3        32
4        30
6        47
         ..
29156    32
29158    27
29159    23
29160    23
29161    24
Name: age, Length: 22402, dtype: int64
0              35
1              35
2              35
3              35
5              35
           ...   
533765         20
533766         20
533767         20
533768         20
533769         20
Name: antiguedad, Length: 424488, dtype: object
0        114
1        114
3         19
4        114
6        114
        ... 
29156     13
29158     33
29159     33
29160     33
29161     33
Name: antiguedad, Length: 22402, dtype: int64
0         1
1         1
2         1
3         1
5         1
         ..
533765    1
533766    1
533767    1
533768    1
533769    1
Name: indrel_1mes, Length: 424488, dtype: object
0        1.0
1    

In [12]:
#将age, antiguedad在train中变为dtype: int64
dtrain_d['age'] = dtrain_d['age'].astype(str).astype(int)
dtrain_d['antiguedad'] = dtrain_d['antiguedad'].astype(str).astype(int)
dtrain_d['indrel_1mes'] = dtrain_d['indrel_1mes'].astype(str).astype(float)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

## 2.2 fecha_dato, ind_empleado, pais_residencia, sexo, fecha_alta, tiprel_1mes, indresi,indext, canal_entrada, indfall, nomprov, segmento

In [13]:
#fecha_dato,fecha_alta, 比较特别，要转化成数字
#print(dtrain_d['fecha_alta'])
#print(dtest_d['fecha_alta'])
#print(dtrain_d['fecha_alta'].unique())

from datetime import datetime

def to_integer(now):
    year = now.strftime("%Y")
    month = now.strftime("%m")
    day = now.strftime("%d")
    return 10000*now.year + 100*now.month + now.day

dtrain_d['fecha_dato'] = pd.to_datetime(dtrain_d['fecha_dato'], format='%Y-%m-%d').apply(lambda x: to_integer(x))
#print(dtrain_d['fecha_dato'])
min_dato = min(dtrain_d['fecha_dato'])
#print(min_dato)
dtrain_d['fecha_dato'] = dtrain_d['fecha_dato'].apply(lambda x: x-min_dato)
#print(dtrain_d['fecha_dato'])

dtest_d['fecha_dato'] = pd.to_datetime(dtest_d['fecha_dato'], format='%Y-%m-%d').apply(lambda x: to_integer(x))
#print(dtest_d['fecha_dato'])
dtest_d['fecha_dato'] = dtest_d['fecha_dato'].apply(lambda x: x-min_dato)
#print(dtrain_d['fecha_dato'])

dtrain_d['fecha_alta'] = pd.to_datetime(dtrain_d['fecha_alta'], format='%Y-%m-%d').apply(lambda x: to_integer(x))
#print(dtrain_d['fecha_alta'])
min_alta = min(dtrain_d['fecha_alta'])
dtrain_d['fecha_alta'] = dtrain_d['fecha_alta'].apply(lambda x: x-min_alta)
#print(dtrain_d['fecha_alta'])

dtest_d['fecha_alta'] = pd.to_datetime(dtest_d['fecha_alta'], format='%Y-%m-%d').apply(lambda x: to_integer(x))
#print(dtest_d['fecha_dato'])
dtest_d['fecha_alta'] = dtest_d['fecha_alta'].apply(lambda x: x-min_dato)
#print(dtrain_d['fecha_dato'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pand

In [14]:
from sklearn import preprocessing
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import LabelEncoder
onc = preprocessing.OneHotEncoder()
lac = preprocessing.LabelEncoder()

In [15]:
#ind_empleado--有5个不同变量
print(dtrain_d['ind_empleado'])
print(dtest_d['ind_empleado'])
print(dtrain_d['ind_empleado'].unique())
# Label-Encoder
lac.fit(dtrain_d['ind_empleado'])
dtrain_d['ind_empleado'] = lac.transform(dtrain_d['ind_empleado'])
lac.fit(dtest_d['ind_empleado'])
dtest_d['ind_empleado'] = lac.transform(dtest_d['ind_empleado'])
print(dtrain_d['ind_empleado'])
print(dtest_d['ind_empleado'])

0         N
1         N
2         N
3         N
5         N
         ..
533765    N
533766    N
533767    N
533768    N
533769    N
Name: ind_empleado, Length: 424488, dtype: object
0        N
1        N
3        N
4        N
6        N
        ..
29156    N
29158    N
29159    N
29160    N
29161    N
Name: ind_empleado, Length: 22402, dtype: object
['N' 'A' 'F' 'B' 'S']
0         3
1         3
2         3
3         3
5         3
         ..
533765    3
533766    3
533767    3
533768    3
533769    3
Name: ind_empleado, Length: 424488, dtype: int64
0        3
1        3
3        3
4        3
6        3
        ..
29156    3
29158    3
29159    3
29160    3
29161    3
Name: ind_empleado, Length: 22402, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [16]:
#pais_residencia--有3种
print(dtrain_d['pais_residencia'])
print(dtest_d['pais_residencia'])
print(dtrain_d['pais_residencia'].unique())
# Label-Encoder
lac.fit(dtrain_d['pais_residencia'])
dtrain_d['pais_residencia'] = lac.transform(dtrain_d['pais_residencia'])
lac.fit(dtest_d['pais_residencia'])
dtest_d['pais_residencia'] = lac.transform(dtest_d['pais_residencia'])
print(dtrain_d['pais_residencia'])
print(dtest_d['pais_residencia'])

0         ES
1         ES
2         ES
3         ES
5         ES
          ..
533765    ES
533766    ES
533767    ES
533768    ES
533769    ES
Name: pais_residencia, Length: 424488, dtype: object
0        ES
1        ES
3        ES
4        ES
6        ES
         ..
29156    ES
29158    ES
29159    ES
29160    ES
29161    ES
Name: pais_residencia, Length: 22402, dtype: object
['ES' 'IT' 'PY']
0         0
1         0
2         0
3         0
5         0
         ..
533765    0
533766    0
533767    0
533768    0
533769    0
Name: pais_residencia, Length: 424488, dtype: int64
0        0
1        0
3        0
4        0
6        0
        ..
29156    0
29158    0
29159    0
29160    0
29161    0
Name: pais_residencia, Length: 22402, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [17]:
#sexo--2种
print(dtrain_d['sexo'])
print(dtest_d['sexo'])
print(dtrain_d['sexo'].unique())
# Label-Encoder
lac.fit(dtrain_d['sexo'])
dtrain_d['sexo'] = lac.transform(dtrain_d['sexo'])
lac.fit(dtest_d['sexo'])
dtest_d['sexo'] = lac.transform(dtest_d['sexo'])
print(dtrain_d['sexo'])
print(dtest_d['sexo'])

0         V
1         V
2         H
3         H
5         H
         ..
533765    V
533766    V
533767    V
533768    V
533769    V
Name: sexo, Length: 424488, dtype: object
0        H
1        H
3        V
4        V
6        V
        ..
29156    H
29158    V
29159    V
29160    H
29161    H
Name: sexo, Length: 22402, dtype: object
['V' 'H']
0         1
1         1
2         0
3         0
5         0
         ..
533765    1
533766    1
533767    1
533768    1
533769    1
Name: sexo, Length: 424488, dtype: int64
0        0
1        0
3        1
4        1
6        1
        ..
29156    0
29158    1
29159    1
29160    0
29161    0
Name: sexo, Length: 22402, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [18]:
#tiprel_1mes--3种
print(dtrain_d['tiprel_1mes'])
print(dtest_d['tiprel_1mes'])
print(dtrain_d['tiprel_1mes'].unique())
# Label-Encoder
lac.fit(dtrain_d['tiprel_1mes'])
dtrain_d['tiprel_1mes'] = lac.transform(dtrain_d['tiprel_1mes'])
lac.fit(dtest_d['tiprel_1mes'])
dtest_d['tiprel_1mes'] = lac.transform(dtest_d['tiprel_1mes'])
print(dtrain_d['tiprel_1mes'])
print(dtest_d['tiprel_1mes'])

0         A
1         A
2         A
3         A
5         I
         ..
533765    I
533766    I
533767    A
533768    A
533769    I
Name: tiprel_1mes, Length: 424488, dtype: object
0        A
1        A
3        I
4        A
6        A
        ..
29156    A
29158    I
29159    A
29160    I
29161    I
Name: tiprel_1mes, Length: 22402, dtype: object
['A' 'I' 'P']
0         0
1         0
2         0
3         0
5         1
         ..
533765    1
533766    1
533767    0
533768    0
533769    1
Name: tiprel_1mes, Length: 424488, dtype: int64
0        0
1        0
3        1
4        0
6        0
        ..
29156    0
29158    1
29159    0
29160    1
29161    1
Name: tiprel_1mes, Length: 22402, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [19]:
#indresi--2
print(dtrain_d['indresi'])
print(dtest_d['indresi'])
print(dtrain_d['indresi'].unique())
# Label-Encoder
lac.fit(dtrain_d['indresi'])
dtrain_d['indresi'] = lac.transform(dtrain_d['indresi'])
lac.fit(dtest_d['indresi'])
dtest_d['indresi'] = lac.transform(dtest_d['indresi'])
print(dtrain_d['indresi'])
print(dtest_d['indresi'])

0         S
1         S
2         S
3         S
5         S
         ..
533765    S
533766    S
533767    S
533768    S
533769    S
Name: indresi, Length: 424488, dtype: object
0        S
1        S
3        S
4        S
6        S
        ..
29156    S
29158    S
29159    S
29160    S
29161    S
Name: indresi, Length: 22402, dtype: object
['S' 'N']
0         1
1         1
2         1
3         1
5         1
         ..
533765    1
533766    1
533767    1
533768    1
533769    1
Name: indresi, Length: 424488, dtype: int64
0        0
1        0
3        0
4        0
6        0
        ..
29156    0
29158    0
29159    0
29160    0
29161    0
Name: indresi, Length: 22402, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [20]:
#indext,---2
print(dtrain_d['indext'])
print(dtest_d['indext'])
print(dtrain_d['indext'].unique())
# Label-Encoder
lac.fit(dtrain_d['indext'])
dtrain_d['indext'] = lac.transform(dtrain_d['indext'])
lac.fit(dtest_d['indext'])
dtest_d['indext'] = lac.transform(dtest_d['indext'])
print(dtrain_d['indext'])
print(dtest_d['indext'])

0         N
1         N
2         N
3         N
5         S
         ..
533765    N
533766    N
533767    N
533768    N
533769    N
Name: indext, Length: 424488, dtype: object
0        N
1        N
3        N
4        N
6        S
        ..
29156    N
29158    N
29159    N
29160    N
29161    N
Name: indext, Length: 22402, dtype: object
['N' 'S']
0         0
1         0
2         0
3         0
5         1
         ..
533765    0
533766    0
533767    0
533768    0
533769    0
Name: indext, Length: 424488, dtype: int64
0        0
1        0
3        0
4        0
6        1
        ..
29156    0
29158    0
29159    0
29160    0
29161    0
Name: indext, Length: 22402, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [21]:
#canal_entrada, 
print(dtrain_d['canal_entrada'])
print(dtest_d['canal_entrada'])
print(dtrain_d['canal_entrada'].unique())
# Label-Encoder
lac.fit(dtrain_d['canal_entrada'])
dtrain_d['canal_entrada'] = lac.transform(dtrain_d['canal_entrada'])
lac.fit(dtest_d['canal_entrada'])
dtest_d['canal_entrada'] = lac.transform(dtest_d['canal_entrada'])
print(dtrain_d['canal_entrada'])
print(dtest_d['canal_entrada'])

0         KFC
1         KAT
2         KFC
3         KHE
5         KHE
         ... 
533765    KFC
533766    KFC
533767    KAT
533768    KFC
533769    KFC
Name: canal_entrada, Length: 424488, dtype: object
0        KBF
1        KAT
3        KHE
4        KFC
6        KFA
        ... 
29156    KFA
29158    KHE
29159    KHE
29160    KHE
29161    KHE
Name: canal_entrada, Length: 22402, dtype: object
['KFC' 'KAT' 'KHE' 'KHD' 'KFA' 'KHC' 'KAZ' 'RED' 'KHK' 'KEH' 'KFJ' 'KFG'
 'KFD' 'KAG' 'KAB' 'KAQ' 'KAE' 'KFU' 'KAH' 'KAA' 'KFF' 'KFP' 'KAR' 'KGX'
 'KCC' 'KGV' 'KFK' 'KHF' 'KHL' 'KBG' 'KBQ' 'KAD' 'KCG' 'KAC' 'KBZ' 'KAY'
 'KBU' 'KCA' 'KCB' 'KDY' 'KCD' 'KEA' 'KAF' 'KDR' '007' 'KES' 'KBO' 'KCL'
 '013' 'KBH' 'KDT' 'KCH' 'KDQ' 'KEI' 'KCN' 'KAS' 'KAJ' 'KDO' 'KCM' 'KBV'
 'KBR' 'KFT' 'KCU' 'KDP' 'KDX' 'KBF' 'KAW' 'KBE' 'KDD' 'KDH' 'KBY' 'KEF'
 'KBW' 'KDC' 'KDS' 'KCP' 'KDF' 'KEY' 'KEZ' 'KAP' 'KFI' 'KDM' 'KEL' 'KEV'
 'KEG' 'KEW' 'KCI' 'KBB' 'KAU' 'KAM' 'K00' 'KAO' 'KAI' 'KFS' 'KAL' 'KEJ'
 'KEN' 'KFN' 'KFH'

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [22]:
#indfall---2
print(dtrain_d['indfall'])
print(dtest_d['indfall'])
print(dtrain_d['indfall'].unique())
# Label-Encoder
lac.fit(dtrain_d['indfall'])
dtrain_d['indfall'] = lac.transform(dtrain_d['indfall'])
lac.fit(dtest_d['indfall'])
dtest_d['indfall'] = lac.transform(dtest_d['indfall'])
print(dtrain_d['indfall'])
print(dtest_d['indfall'])

0         N
1         N
2         N
3         N
5         N
         ..
533765    N
533766    N
533767    N
533768    N
533769    N
Name: indfall, Length: 424488, dtype: object
0        N
1        N
3        N
4        N
6        N
        ..
29156    N
29158    N
29159    N
29160    N
29161    N
Name: indfall, Length: 22402, dtype: object
['N' 'S']
0         0
1         0
2         0
3         0
5         0
         ..
533765    0
533766    0
533767    0
533768    0
533769    0
Name: indfall, Length: 424488, dtype: int64
0        0
1        0
3        0
4        0
6        0
        ..
29156    0
29158    0
29159    0
29160    0
29161    0
Name: indfall, Length: 22402, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [23]:
#nomprov, 
print(dtrain_d['nomprov'])
print(dtest_d['nomprov'])
print(dtrain_d['nomprov'].unique())
# Label-Encoder
lac.fit(dtrain_d['nomprov'])
dtrain_d['nomprov'] = lac.transform(dtrain_d['nomprov'])
lac.fit(dtest_d['nomprov'])
dtest_d['nomprov'] = lac.transform(dtest_d['nomprov'])
print(dtrain_d['nomprov'])
print(dtest_d['nomprov'])

0              TOLEDO
1              MADRID
2          VALLADOLID
3           BARCELONA
5              TOLEDO
             ...     
533765     VALLADOLID
533766      BARCELONA
533767      CANTABRIA
533768      BARCELONA
533769    PALMAS, LAS
Name: nomprov, Length: 424488, dtype: object
0           MADRID
1        BARCELONA
3          BADAJOZ
4           MADRID
6           MADRID
           ...    
29156       MADRID
29158     VALENCIA
29159     ALICANTE
29160     ALICANTE
29161     ZARAGOZA
Name: nomprov, Length: 22402, dtype: object
['TOLEDO' 'MADRID' 'VALLADOLID' 'BARCELONA' 'CIUDAD REAL' 'ZAMORA'
 'VALENCIA' 'SALAMANCA' 'ZARAGOZA' 'CUENCA' 'BADAJOZ' 'CORUÑA, A'
 'CACERES' 'LERIDA' 'CORDOBA' 'MALAGA' 'ASTURIAS' 'SANTA CRUZ DE TENERIFE'
 'CADIZ' 'LEON' 'JAEN' 'SEVILLA' 'SEGOVIA' 'PALENCIA' 'BALEARS, ILLES'
 'CASTELLON' 'LUGO' 'CANTABRIA' 'HUESCA' 'RIOJA, LA' 'OURENSE'
 'PONTEVEDRA' 'TARRAGONA' 'ALICANTE' 'ALBACETE' 'MURCIA' 'GIRONA' 'BURGOS'
 'PALMAS, LAS' 'HUELVA' 'GRANADA' 'AVILA' '

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [24]:
#segmento---3
print(dtrain_d['segmento'])
print(dtest_d['segmento'])
print(dtrain_d['segmento'].unique())
# Label-Encoder
lac.fit(dtrain_d['segmento'])
dtrain_d['segmento'] = lac.transform(dtrain_d['segmento'])
lac.fit(dtest_d['segmento'])
dtest_d['segmento'] = lac.transform(dtest_d['segmento'])
print(dtrain_d['segmento'])
print(dtest_d['segmento'])

0          02 - PARTICULARES
1          02 - PARTICULARES
2         03 - UNIVERSITARIO
3         03 - UNIVERSITARIO
5         03 - UNIVERSITARIO
                 ...        
533765     02 - PARTICULARES
533766     02 - PARTICULARES
533767     02 - PARTICULARES
533768              01 - TOP
533769     02 - PARTICULARES
Name: segmento, Length: 424488, dtype: object
0         02 - PARTICULARES
1                  01 - TOP
3        03 - UNIVERSITARIO
4         02 - PARTICULARES
6         02 - PARTICULARES
                ...        
29156     02 - PARTICULARES
29158    03 - UNIVERSITARIO
29159    03 - UNIVERSITARIO
29160    03 - UNIVERSITARIO
29161    03 - UNIVERSITARIO
Name: segmento, Length: 22402, dtype: object
['02 - PARTICULARES' '03 - UNIVERSITARIO' '01 - TOP']
0         1
1         1
2         2
3         2
5         2
         ..
533765    1
533766    1
533767    1
533768    0
533769    1
Name: segmento, Length: 424488, dtype: int64
0        1
1        0
3        2
4        1
6      

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [25]:
dtrain_d.describe()

,ID,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
count,424488.000000,424488.000000,4.244880e+05,424488.000000,424488.000000,424488.000000,424488.000000,424488.000000,424488.000000,424488.000000,...,424488.000000,424488.000000,424488.000000,424488.000000,424488.000000,424488.000000,424488.000000,424488.000000,424488.000000,424488.000000
mean,261994.946818,3254.845131,7.939328e+05,2.999430,0.000108,0.546715,40.905811,131321.933525,0.025878,84.526780,...,0.006372,0.010940,0.002853,0.060105,0.048246,0.028967,0.003847,0.059097,0.065001,0.139707
std,154069.917856,4292.128980,4.218502e+05,0.037654,0.013380,0.497814,17.188654,55215.972355,0.158772,66.497437,...,0.079573,0.104022,0.053336,0.237682,0.214287,0.167713,0.061905,0.235807,0.246527,0.346683
min,0.000000,0.000000,1.607400e+04,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,128439.750000,400.000000,4.160480e+05,3.000000,0.000000,0.000000,25.000000,80780.000000,0.000000,26.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,257728.500000,800.000000,8.846780e+05,3.000000,0.000000,1.000000,40.000000,150788.000000,0.000000,56.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,395935.250000,10000.000000,1.160219e+06,3.000000,0.000000,1.000000,51.000000,180587.000000,0.000000,142.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,533769.000000,10300.000000,1.454511e+06,4.000000,2.000000,1.000000,111.000000,200695.000000,1.000000,255.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
dtrain_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424488 entries, 0 to 533769
Data columns (total 47 columns):
ID                       424488 non-null int64
fecha_dato               424488 non-null int64
ncodpers                 424488 non-null int64
ind_empleado             424488 non-null int64
pais_residencia          424488 non-null int64
sexo                     424488 non-null int64
age                      424488 non-null int64
fecha_alta               424488 non-null int64
ind_nuevo                424488 non-null float64
antiguedad               424488 non-null int64
indrel                   424488 non-null float64
indrel_1mes              424488 non-null float64
tiprel_1mes              424488 non-null int64
indresi                  424488 non-null int64
indext                   424488 non-null int64
canal_entrada            424488 non-null int64
indfall                  424488 non-null int64
tipodom                  424488 non-null float64
cod_prov                 424488 non

In [27]:
dtest_d.describe()

,ID,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,...,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
count,22402.000000,22402.0,2.240200e+04,22402.000000,22402.0,22402.000000,22402.000000,22402.000000,22402.000000,22402.000000,...,22402.0,22402.000000,22402.000000,22402.000000,22402.0,22402.000000,22402.000000,22402.000000,2.240200e+04,22402.000000
mean,14109.485091,10400.0,8.149308e+05,2.999464,0.0,0.540800,40.506964,-66150.769038,0.000312,88.386394,...,0.0,0.038702,75.041871,0.002009,1.0,26.494063,27.184403,0.564726,1.359744e+05,1.326087
std,8617.945072,0.0,4.270239e+05,0.037792,0.0,0.498344,17.042275,55552.322383,0.017675,66.827488,...,0.0,0.192888,37.860349,0.044775,0.0,12.516944,13.110113,0.495804,3.493697e+05,0.571754
min,0.000000,10400.0,1.607400e+04,0.000000,0.0,0.000000,3.000000,-199905.000000,0.000000,2.000000,...,0.0,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,0.000000,6.773130e+03,0.000000
25%,6758.250000,10400.0,4.295795e+05,3.000000,0.0,0.000000,25.000000,-118897.000000,0.000000,31.000000,...,0.0,0.000000,22.000000,0.000000,1.0,15.000000,17.000000,0.000000,6.877432e+04,1.000000
50%,13357.500000,10400.0,9.250925e+05,3.000000,0.0,1.000000,39.000000,-39323.000000,0.000000,57.000000,...,0.0,0.000000,88.000000,0.000000,1.0,28.000000,30.000000,1.000000,1.013754e+05,1.000000
75%,22402.750000,10400.0,1.180968e+06,3.000000,0.0,1.000000,51.000000,-19202.000000,0.000000,146.000000,...,0.0,0.000000,107.000000,0.000000,1.0,33.000000,33.000000,1.000000,1.550911e+05,2.000000
max,29161.000000,10400.0,1.454511e+06,4.000000,0.0,1.000000,110.000000,790.000000,1.000000,256.000000,...,0.0,1.000000,116.000000,1.000000,1.0,52.000000,50.000000,1.000000,2.889440e+07,2.000000


## 3. Targets 探究

In [28]:
targets = dtrain_d.columns[24:]
print(targets)

Index(['ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
       'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
       'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1',
       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1',
       'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
       'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1',
       'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')


In [29]:
import matplotlib.pyplot as plt

i = 0
dtrain_d.insert(0, 'Target', 0)
for t in targets:
    dtrain_d['Target'] = dtrain_d['Target'] + dtrain_d[t]
    #i = i+1
    #dtrain_d[t]*i
print(dtrain_d['Target'])
plt.plot(range(len(dtrain_d)), dtrain_d['Target'])
plt.show()

0         2.0
1         3.0
2         4.0
3         1.0
5         1.0
         ... 
533765    1.0
533766    0.0
533767    0.0
533768    2.0
533769    0.0
Name: Target, Length: 424488, dtype: float64


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<Figure size 640x480 with 1 Axes>

In [30]:
dtrain_d[['Target']].mean(axis=0)

Target    1.542312
dtype: float64

在train中，我们发现有24个target指标
但每个用户平均不止满足一个指标；从图中可以看到有些用户最多满足12个指标

## 程序合并

In [31]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn import preprocessing
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import LabelEncoder
onc = preprocessing.OneHotEncoder()
lac = preprocessing.LabelEncoder()

## ---------------加入ID行----------------
#这样在某些模型中如果希望用含有NAN的数据，可以根据ID行快速找到
data_train = pd.read_csv('/Users/huangzhan/Documents/开课吧/RS/Chap4.1-预测全家桶与机器学习四大神器/L12/santander-product-recommendation/train.csv',index_col=None)
data_train.insert(0, 'ID', range(len(data_train)))

data_test = pd.read_csv('/Users/huangzhan/Documents/开课吧/RS/Chap4.1-预测全家桶与机器学习四大神器/L12/santander-product-recommendation/test.csv',index_col=None)
data_test.insert(0, 'ID', range(len(data_test)))

## ---------------处理NAN----------------
#删去ult_fec_cli_1t、conyuemp两列
del data_train['ult_fec_cli_1t']
del data_test['ult_fec_cli_1t']
del data_train['conyuemp']
del data_test['conyuemp']

#删去有NAN的行
dtrain_d = data_train.dropna(axis = 0)
dtrain_d.info()
dtest_d = data_test.dropna(axis = 0)
dtest_d.info()

## ---------------数据预处理----------------
#将age, antiguedad在train中变为dtype: int64
dtrain_d['age'] = dtrain_d['age'].astype(str).astype(int)
dtrain_d['antiguedad'] = dtrain_d['antiguedad'].astype(str).astype(int)
dtrain_d['indrel_1mes'] = dtrain_d['indrel_1mes'].astype(str).astype(float)

#fecha_dato,fecha_alta, 比较特别，要转化成数字
def to_integer(now):
    year = now.strftime("%Y")
    month = now.strftime("%m")
    day = now.strftime("%d")
    return 10000*now.year + 100*now.month + now.day

dtrain_d['fecha_dato'] = pd.to_datetime(dtrain_d['fecha_dato'], format='%Y-%m-%d').apply(lambda x: to_integer(x))
min_dato = min(dtrain_d['fecha_dato'])
dtrain_d['fecha_dato'] = dtrain_d['fecha_dato'].apply(lambda x: x-min_dato)

dtest_d['fecha_dato'] = pd.to_datetime(dtest_d['fecha_dato'], format='%Y-%m-%d').apply(lambda x: to_integer(x))
dtest_d['fecha_dato'] = dtest_d['fecha_dato'].apply(lambda x: x-min_dato)

dtrain_d['fecha_alta'] = pd.to_datetime(dtrain_d['fecha_alta'], format='%Y-%m-%d').apply(lambda x: to_integer(x))
min_alta = min(dtrain_d['fecha_alta'])
dtrain_d['fecha_alta'] = dtrain_d['fecha_alta'].apply(lambda x: x-min_alta)

dtest_d['fecha_alta'] = pd.to_datetime(dtest_d['fecha_alta'], format='%Y-%m-%d').apply(lambda x: to_integer(x))
dtest_d['fecha_alta'] = dtest_d['fecha_alta'].apply(lambda x: x-min_dato)

#ind_empleado, pais_residencia, sexo, tiprel_1mes, indresi,indext, canal_entrada, indfall, nomprov, segmento
lac.fit(dtrain_d['ind_empleado'])
dtrain_d['ind_empleado'] = lac.transform(dtrain_d['ind_empleado'])
lac.fit(dtest_d['ind_empleado'])
dtest_d['ind_empleado'] = lac.transform(dtest_d['ind_empleado'])

lac.fit(dtrain_d['pais_residencia'])
dtrain_d['pais_residencia'] = lac.transform(dtrain_d['pais_residencia'])
lac.fit(dtest_d['pais_residencia'])
dtest_d['pais_residencia'] = lac.transform(dtest_d['pais_residencia'])

lac.fit(dtrain_d['sexo'])
dtrain_d['sexo'] = lac.transform(dtrain_d['sexo'])
lac.fit(dtest_d['sexo'])
dtest_d['sexo'] = lac.transform(dtest_d['sexo'])

lac.fit(dtrain_d['tiprel_1mes'])
dtrain_d['tiprel_1mes'] = lac.transform(dtrain_d['tiprel_1mes'])
lac.fit(dtest_d['tiprel_1mes'])
dtest_d['tiprel_1mes'] = lac.transform(dtest_d['tiprel_1mes'])

lac.fit(dtrain_d['indresi'])
dtrain_d['indresi'] = lac.transform(dtrain_d['indresi'])
lac.fit(dtest_d['indresi'])
dtest_d['indresi'] = lac.transform(dtest_d['indresi'])

lac.fit(dtrain_d['indext'])
dtrain_d['indext'] = lac.transform(dtrain_d['indext'])
lac.fit(dtest_d['indext'])
dtest_d['indext'] = lac.transform(dtest_d['indext'])

lac.fit(dtrain_d['canal_entrada'])
dtrain_d['canal_entrada'] = lac.transform(dtrain_d['canal_entrada'])
lac.fit(dtest_d['canal_entrada'])
dtest_d['canal_entrada'] = lac.transform(dtest_d['canal_entrada'])

lac.fit(dtrain_d['indfall'])
dtrain_d['indfall'] = lac.transform(dtrain_d['indfall'])
lac.fit(dtest_d['indfall'])
dtest_d['indfall'] = lac.transform(dtest_d['indfall'])

lac.fit(dtrain_d['nomprov'])
dtrain_d['nomprov'] = lac.transform(dtrain_d['nomprov'])
lac.fit(dtest_d['nomprov'])
dtest_d['nomprov'] = lac.transform(dtest_d['nomprov'])

lac.fit(dtrain_d['segmento'])
dtrain_d['segmento'] = lac.transform(dtrain_d['segmento'])
lac.fit(dtest_d['segmento'])
dtest_d['segmento'] = lac.transform(dtest_d['segmento'])

## ---------------train中分离target指标----------------
Target_tr = pd.DataFrame(columns = ["ID"]) #创建一个dataframe存放train中的Target
Target_tr['ID'] = dtrain_d['ID']

targets = dtrain_d.columns[25:]
for t in targets:
    Target_tr.insert(0, t, dtrain_d[t])
    del dtrain_d[t]
#dtrain_d.info
Target_tr.info

## ---------------对每一个targets分别使用模型----------------

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 424488 entries, 0 to 533769
Data columns (total 47 columns):
ID                       424488 non-null int64
fecha_dato               424488 non-null object
ncodpers                 424488 non-null int64
ind_empleado             424488 non-null object
pais_residencia          424488 non-null object
sexo                     424488 non-null object
age                      424488 non-null object
fecha_alta               424488 non-null object
ind_nuevo                424488 non-null float64
antiguedad               424488 non-null object
indrel                   424488 non-null float64
indrel_1mes              424488 non-null object
tiprel_1mes              424488 non-null object
indresi                  424488 non-null object
indext                   424488 non-null object
canal_entrada            424488 non-null object
indfall                  424488 non-null object
tipodom                  424488 non-null float64
cod_prov                

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<bound method DataFrame.info of         ind_recibo_ult1  ind_nom_pens_ult1  ind_nomina_ult1  ind_viv_fin_ult1  \
0                     0                0.0              0.0                 0   
1                     0                0.0              0.0                 0   
2                     1                0.0              0.0                 0   
3                     0                0.0              0.0                 0   
5                     0                0.0              0.0                 0   
...                 ...                ...              ...               ...   
533765                0                0.0              0.0                 0   
533766                0                0.0              0.0                 0   
533767                0                0.0              0.0                 0   
533768                0                0.0              0.0                 0   
533769                0                0.0              0.0                 0

In [ ]:
#pip install lightgbm
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split

#创建Target_te存放预测后test集的数据
Target_te = pd.DataFrame(columns = ["ID"]) #创建一个dataframe存放train中的Target
Target_te['ID'] = dtest_d['ID']

targets = Target_tr.columns[:]
for t in targets:
    X_train, X_valid, y_train, y_valid = train_test_split(dtrain_d, Target_tr[t], test_size=0.3, random_state=42)
    
    # 创建模型，训练模型
    gbm = lgb.LGBMClassifier(silent=True,num_leaves=51,max_depth=10,boosting_type='gbdt',
                             objective='binary',learning_rate=0.015,n_estimators=1000,
                             subsample=0.75,subsample_freq=2,reg_lambda=0.28,reg_alpha=0.12,
                             colsample_bytree=0.8,min_child_samples=300,min_split_gain=0.1)
    
    gbm.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_valid, y_valid)],eval_metric='auc', 
            early_stopping_rounds=100)
    # 预测数据集
    prob = gbm.predict_proba(test)
    Target_te.insert(0, t, pd.Series(prob[:,1]))
Target_te

In [ ]:
import catboost as cb
import numpy as np
from sklearn.model_selection import train_test_split

#创建Target_te存放预测后test集的数据
Target_te = pd.DataFrame(columns = ["ID"]) #创建一个dataframe存放train中的Target
Target_te['ID'] = dtest_d['ID']

targets = Target_tr.columns[:]
for t in targets:
    X_train, X_valid, y_train, y_valid = train_test_split(dtrain_d, Target_tr[t], test_size=0.3, random_state=42)
    
    model = cb.CatBoostClassifier(iterations=1000, 
                              depth=7, 
                              learning_rate=0.01, 
                              loss_function='Logloss', 
                              eval_metric='AUC',
                              logging_level='Verbose', 
                              metric_period=50
                             )

    # 得到分类特征的列号
    categorical_features_indices = []
    for i in range(len(X_train.columns)):
        if X_train.columns.values[i] in attr:
            categorical_features_indices.append(i)
    print(categorical_features_indices)

    model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=categorical_features_indices)

    #model = cb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
    #predict = model.predict(test)
    #predict = model.predict_proba(test)
    #print(predict)
    # 预测数据集
    prob = model.predict_proba(test)
    Target_te.insert(0, t, pd.Series(prob[:,1]))
Target_te
    ## 转化为二分类输出
    #test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
    #test[['Attrition']].to_csv('submit_cb.csv')

